## Generating a System parametrized with a SMIRNOFF force field

This example shows how to generate parameterized `System` object from a SMIRNOFF force field.

Note: This example is adopted from a similar example in the Open Force Field toolkit:  a simulation of a System parametrized with a SMIRNOFF force field. See [here](https://github.com/openforcefield/openff-toolkit/tree/3de67ca96265a3953c008e8e244c380d4aa81f2e/examples/SMIRNOFF_simulation) or a more recent version of the same example.

In [ ]:
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.utils import get_data_file_path
from simtk import unit as omm_unit
from simtk.openmm.app import PDBFile

from openff.system.stubs import ForceField

# First, grab some reference structures from the toolkit's test suite
pdb_file_path = get_data_file_path("systems/test_systems/1_cyclohexane_1_ethanol.pdb")
pdbfile = PDBFile(pdb_file_path)

ethanol = Molecule.from_smiles("CCO")
cyclohexane = Molecule.from_smiles("C1CCCCC1")

# Create the Open Force Field Topology from an OpenMM Topology object.
omm_topology = pdbfile.topology
off_topology = Topology.from_openmm(
    omm_topology, unique_molecules=[ethanol, cyclohexane]
)

# Load the OpenFF "Parsley" force field.
parsley = ForceField("openff-1.0.0.offxml")

In [ ]:
# Create an OpenFF System object from the force field and topology objects
openff_sys = parsley.create_openff_system(topology=off_topology)

In [ ]:
# Convert the OpenFF System to an OpenMM System
openmm_sys = openff_sys.to_openmm(combine_nonbonded_forces=True)


### Run a simulation

We can now use the `System` object to run molecular dynamics trajectories with OpenMM.

In [ ]:
from simtk import openmm, unit

# Propagate the System with Langevin dynamics.
time_step = 2 * unit.femtoseconds  # simulation timestep
temperature = 300 * unit.kelvin  # simulation temperature
friction = 1 / unit.picosecond  # collision rate
integrator = openmm.LangevinIntegrator(temperature, friction, time_step)

# Length of the simulation.
num_steps = 1000  # number of integration steps to run

# Logging options.
trj_freq = 1  # number of steps per written trajectory frame
data_freq = 1  # number of steps per written simulation statistics

# Set up an OpenMM simulation.
simulation = openmm.app.Simulation(omm_topology, openmm_sys, integrator)

# Set the initial positions.
positions = pdbfile.getPositions()
simulation.context.setPositions(positions)

# Randomize the velocities from a Boltzmann distribution at a given temperature.
simulation.context.setVelocitiesToTemperature(temperature)

# Configure the information in the output files.
pdb_reporter = openmm.app.PDBReporter("trajectory.pdb", trj_freq)
state_data_reporter = openmm.app.StateDataReporter(
    "data.csv",
    data_freq,
    step=True,
    potentialEnergy=True,
    temperature=True,
    density=True,
)
simulation.reporters.append(pdb_reporter)
simulation.reporters.append(state_data_reporter)

In [ ]:
import time

print("Starting simulation")
start = time.process_time()

# Run the simulation
simulation.step(num_steps)

end = time.process_time()
print("Elapsed time %.2f seconds" % (end - start))
print("Done!")

If successful, the directory where your jupyter notebook is running should contain a `trajectory.pdb` file that you can visualize and a `data.csv` file including potential energy, density, and temperature of each frame.